In [1]:
# from   pathlib      import Path
# import importlib

# import numpy                as np
# import matplotlib           as mpl
# import matplotlib.pyplot    as plt
# import seaborn              as sns
# import pandas               as pd

# from scipy.stats   import kruskal, wilcoxon, mannwhitneyu, ranksums
# from scipy.optimize import curve_fit
# import statsmodels.api as sm
# from statsmodels.multivariate.manova import MANOVA
# import statsmodels.formula.api as smf
# from eidynamics     import utils, plot_tools
# plt.rcParams['font.family'] = 'Arial'
# plt.rcParams['font.size'] = 12
# plt.rcParams['svg.fonttype'] = 'none'

# # make a colour map viridis
# viridis = mpl.colormaps["viridis"]
# flare   = mpl.colormaps["flare"]
# crest   = mpl.colormaps["crest"]
# magma   = mpl.colormaps["magma"]
# edge    = mpl.colormaps['edge']

# color_E = "flare"
# color_I = "crest"
# color_freq = {1:magma(0.05), 5:magma(0.1), 10:magma(0.2), 20:magma(.4), 30:magma(.5), 40:magma(.6), 50:magma(.7), 100:magma(.9)}
# color_squares = color_squares = {1:viridis(0.2), 5:viridis(.4), 7:viridis(.6), 15:viridis(.8), 20:viridis(1.0)}
# color_EI = {-70:flare(0), 0:crest(0)}
# colors_EI = {-70:flare, 0:crest}

# Fs = 2e4
# %matplotlib widget
# freq_sweep_pulses = np.arange(9)

# from eidynamics.fit_PSC     import find_sweep_expected
# # from Findsim        import tab_presyn_patterns_LR_43
# # import parse_data
# from eidynamics     import utils, plot_tools
# import all_cells
# # import plotFig2
# import stat_annotate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import plotFig2
%colors nocolor
Fs=2e4

In [2]:
# load data
figure_raw_material_location = Path(r"paper_figure_matter\\")
data_path                    = Path(r"parsed_data\\FreqSweep\\")

# Load the dataset
freq_sweep_cc_datapath =  Path(r"parsed_data\\FreqSweep\\all_cells_FreqSweep_CC_long.h5")
df = pd.read_hdf(freq_sweep_cc_datapath, key='data')

# # expanded dataframe (processed dataframe with metadata and analysed params)
# expanded_data_path = Path(r"parsed_data\all_cells_FreqSweep_combined_expanded.h5")
# xc_FS_analyseddf = pd.read_hdf(expanded_data_path, key='data')

In [3]:
df.shape

(4971, 80073)

In [ ]:
# remove columns that are not needed
# not needed columns are 20049 to 80049
df = df.drop(df.columns[20049:80049], axis=1)
df.shape

In [ ]:
import importlib
importlib.reload(plotFig2)

new_fields = ['stimtime_','valley_', 'peak_', 'slope_', 'peakdelay_', 'onsetdelay_','normpeak_', 'PSC_', 'normPSC_', 'spike_']
newcolumns = [field + str(i) for field in new_fields for i in range(9)]
Fs=2e4
# Adding new columns with NaN values
df = df.assign(**{col: np.nan for col in newcolumns})
c = 0
d=0
r=0
k=0
for idx, row in df.iterrows():
    print(f"Processing {idx} row with trialID {row['trialID']}")
    # do not process if freq==10
    if row['stimFreq'] <20:
        print(f"Skipping {idx} row with trialID {row['trialID']}: Low frequency")
        d+=0
        continue
    
    try:
        if row['probePulseStart'] == row['pulseTrainStart']:
            freq = row['stimFreq']
            isi = int(0.9*Fs / freq)
            print(f"interpolating {idx} row with trialID {row['trialID']}: No probe pulse. {row['probePulseStart']} =/= { row['pulseTrainStart']}")
            row[2000:2000+isi] = row[4620:4620+isi]
            row[2000+isi-100:4620] = np.interp(np.arange(2000+isi-100,4620), [2000+isi-100,4620], [row[2000+isi-100], row[4620]])
            c+=1
            results = plotFig2.deconv(row[49:80049], row['stimFreq'], 0.1, row['pulseTrainStart'], None, noprobepulse=False)
        else:
            results = plotFig2.deconv(row[49:80049], row['stimFreq'], row['probePulseStart'], row['pulseTrainStart'], None, noprobepulse=False)
    except RuntimeError:
        r+=1
        continue
    except FloatingPointError as e:
        print(f"Error at {idx} row with trialID {row['trialID']}: {e}")
        print(row['stimFreq'], row['numSq'], row['probePulseStart'], row['pulseTrainStart'])
        k+=1
        continue
    
    valleyTimes, valleys, peakTimes, peaks = results[2]
    # elementwise difference between peak and valley is PSC
    pscs = np.array(peaks) - np.array(valleys)
    normPSCs = np.array(pscs) / pscs[0]
    normpeaks = np.array(peaks) / peaks[0]
    stimTimes = np.array(results[-1]) / Fs    
    slopes = np.array([(peaks[i] - valleys[i]) / (peakTimes[i] - valleyTimes[i]) for i in range(len(peaks))])
    peakdelays = np.array([(peakTimes[i] - stimTimes[i]) for i in range(len(valleyTimes))])
    onsetdelays = np.array([(valleyTimes[i] - stimTimes[i]) for i in range(len(valleyTimes))])
    # spikes if peaks are greater than 20
    spikes = np.array([1 if peak > 20 else 0 for peak in peaks])

    for col in newcolumns:
        prefix, col_idx = col.split('_')
        col_idx = int(col_idx)

        if col_idx < len(valleys):  # Ensure index is within bounds
            if prefix == 'stimtime':
                df.loc[row.name, col] = stimTimes[col_idx]
            elif prefix == 'valley':
                df.loc[row.name, col] = valleys[col_idx]
            elif prefix == 'peak':
                df.loc[row.name, col] = peaks[col_idx]
            elif prefix == 'slope':
                df.loc[row.name, col] = slopes[col_idx]
            elif prefix == 'peakdelay':
                df.loc[row.name, col] = peakdelays[col_idx]
            elif prefix == 'onsetdelay':
                df.loc[row.name, col] = onsetdelays[col_idx]
            elif prefix == 'normpeak':
                df.loc[row.name, col] = normpeaks[col_idx]
            elif prefix == 'PSC':
                df.loc[row.name, col] = pscs[col_idx]
            elif prefix == 'normPSC':
                df.loc[row.name, col] = normPSCs[col_idx]
            elif prefix == 'spike':
                df.loc[row.name, col] = spikes[col_idx]


    # counter for every 100 rows
    if idx % 100 == 0:
        print(f"Processed {idx} rows")

# Assuming df is your final DataFrame
metadata_cols1 = df.iloc[:, :49]
metadata_cols2 = df.iloc[:, 20049:]

# Concatenating the two slices into a new DataFrame
new_df = pd.concat([metadata_cols1, metadata_cols2], axis=1)

# Display the shape of the new DataFrame to verify
print(new_df.shape)
print(f"Kernel errors: {k}, Runtime errors: {r}, No probe pulse: {c}, Low frequency: {d}")
4113

In [ ]:
new_df.shape

In [ ]:
print(new_df.shape)
# save df
new_df.to_hdf(data_path / "all_cells_FreqSweep_CC_kernelfit_response_measurements.h5", key='data', mode='w')
# save as excel
new_df.to_excel(data_path / "all_cells_FreqSweep_CC_kernelfit_response_measurements.xlsx")

# drop those rows from df that have NaN values in column 'valley_0'
new_df = new_df.dropna(subset=['valley_0'])
print(new_df.shape)

In [ ]:
# save df
new_df.to_hdf(data_path / "all_cells_FreqSweep_CC_kernelfit_response_measurements_noNANs.h5", key='data', mode='w')
# save as excel
new_df.to_excel(data_path / "all_cells_FreqSweep_CC_kernelfit_response_measurements_noNANs.xlsx")